# Global Sensitivity Analysis

This is a more complex case. Involves varying multiple parameters together and understanding which parameter is the one with the largest influence on results. This translates into a large simulation testing the various possibilities. One could use brute force and test all possible combinations of all parameters, and then run a regression on the results where the dependent variable is the impact of the system and the independent variables are the parameters. However, calculating a result for every possible combination might end up taking too much computation time, especially if the number of tested variables is high. 

## Correlation Analysis
The content below shows how to perform a simple Global Sensitivity Analysis (GSA) for a example product system of a biobased product and using correlation analysis to quantify the defree of sensitivity.

In [1]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
bd.projects.set_current('advlca25') # Still working in the same project
bd.databases

Databases dictionary with 8 object(s):
	ALIGNED-biob-prod-dummy
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb
	lccfg
	lcctestdb
	testbiosphere
	testdb

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [3]:
# Import the dummy product system

# import data from csv

#mydata = pd.read_csv('LCI1-bw-format.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 'Exchange minimum', 'Exchange maximum', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [4]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

14:36:53 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 6683.08it/s]

14:36:53 [info     ] Vacuuming database            


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [5]:
# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2


In [6]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'processwithreferenceproduct',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 158206687701803008}

In [7]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a nominal climate impact score for the fictional biobased product, to be used for reference later on.

In [9]:
# calculation of nominal LCA score
mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

180.28497357367445


### Now perform global sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input, in this case, only 5 values.
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) A correlation is estimated between model input values and output values.

#### Step 1
A sample of values for each parameter.

In [10]:
# Can be done in many ways, here very basic using lists.
par1_values = [-1.25, -1.10, -1, -0.9, -0.75] # In bomass growth, value of CO2 uptake
par2_values = [0.1, 0.3, 0.5, 0.7, 0.9] # In biomass processing, amount of energy used.
par3_values = [20,30,40,50,60]  # In use of product, amount of manufactured product needed
par4_values = [1.25, 1.10, 1, 0.9, 0.75] # In use of product, amount of manufactured product needed

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices

In [11]:
param_samples = [(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'), 
  ('ecoinvent-3.11-biosphere','349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par1_values), # In bomass growth, value of CO2 uptake
(('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
 ('ecoinvent-3.11-consequential','a69490974731f9da34043d5c926cd167'), par2_values), # In biomass processing, amount of energy used.
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'), par3_values), # In use of product, amount of manufactured product needed
  (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
   ('ecoinvent-3.11-biosphere','349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par4_values)] # In end of life, amount of CO2 released

#### (technical note)

We can do the same in a more elegant way, taking the data directly from the BW database that we just created...

In [12]:
# In biomass growth, value of CO2 uptake
par1 = list(bd.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bd.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bd.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = 5
param_samples = [(par1['output'],par1['input'], par1.random_sample(n = n_iter)),
                 (par2['output'],par2['input'], par2.random_sample(n = n_iter)),
                 (par3['output'],par3['input'], par3.random_sample(n = n_iter)),
                 (par4['output'],par4['input'], par4.random_sample(n = n_iter))]

Let's look at the samples obtained

In [13]:
param_samples

[(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('ecoinvent-3.11-biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([-1.10034447, -1.07858308, -0.78085421, -1.07736422, -1.23161972])),
 (('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
  ('ecoinvent-3.11-consequential', 'a69490974731f9da34043d5c926cd167'),
  array([0.49801342, 0.38347052, 0.5516293 , 0.45764931, 0.62401611])),
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
  array([57.07401351, 44.62772923, 59.89983582, 44.52616595, 46.30428387])),
 (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
  ('ecoinvent-3.11-biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([0.96469963, 1.09797637, 1.24865035, 0.96165161, 1.22047394]))]

#### Step 2

Iteration through all parameter samples. Input values are replaced and new impact scores are calculated at each iteration.

In [14]:
# testing the loop that will be used for the simulation, iterate through parameter values
    
for i in range(0,n_iter): # iterate 5 times...
    for s in param_samples: # and for each paramater...
        
        # Get the 'id' using the activity object
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id
        
        # Use the id and the mapping dictionaries to find matrix row and columns
        if s[1][0] == 'ecoinvent-3.11-biosphere':
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.biosphere[row_id] # find row index of B matrix for the exchange
            print(f"biosphere value: {LCA.biosphere_matrix[row,col]} | paramater value: {s[2][i]}")
        else:
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.activity[row_id] # find row index of A matrix for the exchange
            print(f"technosphere value: {LCA.technosphere_matrix[row,col]} | paramater value: {-s[2][i]}") # CHANGE OF SIGN!
        
    print("* * *")

biosphere value: -1.0 | paramater value: -1.100344474923109
technosphere value: -0.5 | paramater value: -0.49801342449399233
technosphere value: -50.0 | paramater value: -57.07401351211584
biosphere value: 1.0 | paramater value: 0.9646996339355052
* * *
biosphere value: -1.0 | paramater value: -1.0785830822698703
technosphere value: -0.5 | paramater value: -0.3834705189433636
technosphere value: -50.0 | paramater value: -44.627729228738104
biosphere value: 1.0 | paramater value: 1.097976368881174
* * *
biosphere value: -1.0 | paramater value: -0.7808542075579623
technosphere value: -0.5 | paramater value: -0.5516292981325623
technosphere value: -50.0 | paramater value: -59.89983581508282
biosphere value: 1.0 | paramater value: 1.2486503519675274
* * *
biosphere value: -1.0 | paramater value: -1.0773642194053594
technosphere value: -0.5 | paramater value: -0.4576493128015625
technosphere value: -50.0 | paramater value: -44.526165947418875
biosphere value: 1.0 | paramater value: 0.961651

In [15]:
# Implementing the loop
import copy # a library helps to save the original matrices

GSA_value_results = []

for i in range(0,n_iter): 
    LCA.lci()
    LCA.lcia()
    print("initial value", LCA.score)
    
    old_bio_matrix = copy.deepcopy(LCA.biosphere_matrix)
    old_tech_matrix = copy.deepcopy(LCA.technosphere_matrix)
    
    for s in param_samples:
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id

        if s[1][0] == 'ecoinvent-3.11-biosphere':
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.biosphere[row_id] # find row index of B matrix for the exchange
            new_bio = s[2][i]
            LCA.biosphere_matrix[row,col] = new_bio # substitute the value
        else:
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.activity[row_id] # find row index of A matrix for the exchange
            new_tech = -s[2][i]
            LCA.technosphere_matrix[row,col] = new_tech # substitute the value
                
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    GSA_value_results.append(LCA.score)
    print('end value', LCA.score)
    print("---")

    # Restore the original matrices
    LCA.biosphere_matrix = old_bio_matrix
    LCA.technosphere_matrix = old_tech_matrix

initial value 180.28497357367445
end value 193.95129634385825
---
initial value 180.28497357367445
end value 169.4361572696431
---
initial value 180.28497357367445
end value 224.94563110865064
---
initial value 180.28497357367445
end value 162.451260206448
---
initial value 180.28497357367445
end value 176.49614662305444
---


#### Step 3

Now we have both the input and output values and we can look at their correlation.

To estimate the degree of correlation use a Pearson coefficient of linear relationship between two sets of values.

See here for info: 
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

This approach has for example been used in:

_Kim, A., Mutel, C., Froemelt, A., 2021. Robust high-dimensional screening. Environmental Modelling & Software 105270._ https://doi.org/10.1016/j.envsoft.2021.105270


In [16]:
# calcualte correlation

corr_data = pd.DataFrame([i[2] for i in param_samples], index = ['par1','par2', 'par3', 'par4']).T
corr_data['GWI'] = GSA_value_results
corr_data.corr()

,par1,par2,par3,par4,GWI
par1,1.000000,-0.065893,0.671864,0.319435,0.781122
par2,-0.065893,1.000000,0.328698,0.583272,0.410900
par3,0.671864,0.328698,1.000000,0.208573,0.940491
par4,0.319435,0.583272,0.208573,1.000000,0.504140
GWI,0.781122,0.410900,0.940491,0.504140,1.000000


In this specific case "par3"  is the parameter to which the results are most sensitive to.

In [17]:
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>

## FAST

The content below shows how to perform a simple Global Sensitivity Analysis (GSA) applying the FAST, a variance-based sensitivity analysis method, for an example product system of a biobased product.

Special thanks to _Pierre Jouannais_ for developing an early version of this tutorial.

In [18]:
# importing packages
import bw2calc as bc
import bw2data as bd
import pandas as pd
import numpy as np
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [19]:
# open a project with ecoinvent v.3.9.1 consequential system model
bd.projects.set_current('advlca25')

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [20]:
# Import the dummy product system

# import data from csv
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 'Exchange minimum', 'Exchange maximum', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [21]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

14:37:19 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 7463.17it/s]

14:37:19 [info     ] Vacuuming database            


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [22]:
# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df


In [23]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'processwithreferenceproduct',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 158206797701619715}

In [24]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [26]:
# calculation of static LCA score
mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

180.28497357367445


### Now perform sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input **using a specific and efficient sampling design** (the **"problem"** below)
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) **A sensitivity index is calculated** using model input values (the "problem") and output values.

#### Step 1
Obtain a sample of values for each parameter.

This is a  pseudo-random sample obtained using a sampling method from the SALib package on the problem previously defined.

it is defined as a __FAST__ sample ("Fourier Amplitude Sensitivity Analysis"), another alternative to Sobol method.

Sobol and FAST sampling methods use different pseudo-random patterns to cover the space of parameters, and different calculations for the sensitivity indices.

Besides these differences, the (extended) FAST method is, just like the Sobol one, a variance-based method for sensitivity analysis. It provides the same information as the Sobol method but with higher computational efficiency.

For further information and comparison see: 

_Saltelli, A.; Tarantola, S.; Chan, K. P. S. A Quantitative Model-Independent Method for Global Sensitivity Analysis of Model Output. Technometrics 1999, 41 (1), 39–56._ https://doi.org/10.1080/00401706.1999.10485594.


Before you run the next cell __you need to install the SALib library!__

Please do `conda install SALib` in the brightway environment´

In [27]:
# need to import the SALib library and relaive methods for sensitvity analysis
# do 'conda install SALib' in the brightway environment

from SALib.sample import saltelli
from SALib.sample import fast_sampler
from SALib.analyze import sobol
from SALib.analyze import fast


Define the "problem" for the GSA analysis as indicated in the SALib library.
Uniform distributions for the uncertainty of for each input parameter are here assumed.

In [28]:
problem = { 'num_vars': 4, # number of variables
            'names': ['par1', 'par2', 'par3', 'par4'], # names of variables, same as parameters
            'bounds': [[-1.25, -0.75], # careful here to what is the lower bound with negative values...
                       [0.1, 0.9],  
                       [20, 60],
                       [0.75, 1.25]] ,
           'dists':["unif","unif","unif","unif"] } # all uniform distributions

In [29]:
# FAST sampler creates N*P parameters sets with N=sample size, P = number of parameters
param_values_FAST = fast_sampler.sample(problem, 200)
print(param_values_FAST.shape)

param_values_FAST

(800, 4)


array([[-0.78929891,  0.83712175, 56.85608747,  1.21070109],
       [-0.90929891,  0.82912175, 56.05608747,  1.19570109],
       [-1.02929891,  0.82112175, 55.25608747,  1.18070109],
       ...,
       [-0.9134298 ,  0.66251231, 49.32561571,  1.0684298 ],
       [-0.9184298 ,  0.64651231, 48.12561571,  1.1884298 ],
       [-0.9234298 ,  0.63051231, 46.92561571,  1.1915702 ]])

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices, taking the data directly from the BW database that we just created.

In [30]:
# In biomass growth, value of CO2 uptake
par1 = list(bd.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bd.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bd.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = len(param_values_FAST)
param_samples = [(par1['output'],par1['input'], [i[0] for i in param_values_FAST]),
                 (par2['output'],par2['input'], [i[1] for i in param_values_FAST]),
                 (par3['output'],par3['input'], [i[2] for i in param_values_FAST]),
                 (par4['output'],par4['input'], [i[3] for i in param_values_FAST])]

#### Step 2

Iteration through all parameter samples. New LCA scores are calculated for each combination of values.

With 800 iterations, **this will take some time**

In [ ]:
# Implementing the loop (This cell will take some time to run)
import copy # a library helps to save the original matrices

GSA_value_results = []

for i in range(0,len(param_values_FAST)): 
    LCA.lci()
    LCA.lcia()
    print("initial value", LCA.score)
    
    old_bio_matrix = copy.deepcopy(LCA.biosphere_matrix)
    old_tech_matrix = copy.deepcopy(LCA.technosphere_matrix)
    
    for s in param_samples:
        # Get the 'id' using the activity object
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id
        
        # Use the id and the mapping dictionaries to find matrix row and columns
        if s[1][0] == 'ecoinvent-3.11-biosphere':
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.biosphere[row_id] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = s[2][i] # substitute the value
        else:
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.activity[row_id] # find row index of A matrix for the exchange
            LCA.technosphere_matrix[row,col] = -s[2][i] # substitute the value
        
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    GSA_value_results.append(LCA.score)
    print('end value', LCA.score)
    print("---")

    # Restore original matrices
    LCA.biosphere_matrix = old_bio_matrix
    LCA.technosphere_matrix = old_tech_matrix

#### Step 3

Now we have both the input and output values and we can feed the problem and the results to the Sobol function to obtain the FAST indices.

First we look at the parameters and the result



In [35]:
# Organize data first to give a look at what we have done: a sample of input values and a corresponding output value
fast_data = pd.DataFrame([i[2] for i in param_samples], index = ['par0','par1', 'par2', 'par3']).T
fast_data['GWI'] = GSA_value_results
fast_data.head(10) # show only the first ten samples

,par0,par1,par2,par3,GWI
0,-0.789299,0.837122,56.856087,1.210701,214.906975
1,-0.909299,0.829122,56.056087,1.195701,208.626490
2,-1.029299,0.821122,55.256087,1.180701,202.442248
3,-1.149299,0.813122,54.456087,1.165701,196.354249
4,-1.230701,0.805122,53.656087,1.150701,191.397996
5,-1.110701,0.797122,52.856087,1.135701,191.829774
6,-0.990701,0.789122,52.056087,1.120701,192.165795
7,-0.870701,0.781122,51.256087,1.105701,192.406059
8,-0.750701,0.773122,50.456087,1.090701,192.550566
9,-0.869299,0.765122,49.656087,1.075701,186.675399


Now we calculate the SI index

In [36]:
si = fast.analyze(problem, np.array(GSA_value_results), print_to_console=True) # must use np.array

            S1        ST   S1_conf   ST_conf
par1  0.008741  0.009749  0.084121  0.079559
par2  0.000032  0.000875  0.074418  0.091467
par3  0.933678  0.936661  0.072216  0.103028
par4  0.055244  0.057548  0.068082  0.090582


In this specific case "par3"  is the parameter to which the results are most sensitive to, this because it has the highest value for "S1" which is the first order effect.

This is determined with relativelygood confidence as the error (value of "S1_conf") is small compared to the coefficient.

Note also that "par3" is also the parameter withi highest sensitivity in the total model, i.e. when in combinatino with other parameters, as observed by the highest value of "ST".

In [37]:
# In this specific case "par3" is the parameter to which the results ar emost sensitive to, and values have high confidente (small error)
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>